## Generate features on all test + train and get predictions
* Todo: seperate feature generation stage, and allow addition of CV models, etc'. 
* uses list of candidates from lightfm
    * preprocess moti's predictions using code in `transform_moti_preds.py`

In [1]:
# !pip install catboost shap

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
from __future__ import print_function, absolute_import

from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor,CatBoost
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import gc
import matplotlib.pyplot as plt
# import shap
# shap.initjs()

from copy import deepcopy
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, GroupShuffleSplit, GridSearchCV, GroupKFold
from tensorflow.keras.metrics import TopKCategoricalAccuracy, Precision, SparseTopKCategoricalAccuracy # @4
from tensorflow.keras.utils import to_categorical
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers

# from sklearn.metrics import top_k_accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np
np.random.seed(0)
# from sklearn.linear_model import  LogisticRegressionCV
# from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# from pytorch_tabnet.tab_model import TabNetClassifier
# from pytorch_tabnet.pretraining import TabNetPretrainer
# from pytorch_tabnet.metrics import Metric
# import torch

import datetime as dt
d1 = dt.datetime(2015,12,30) # arbitrary min start date , use for converting checkin, checkout to numbers in nice format

%matplotlib inline

pd.set_option("display.max_columns", 90)

In [4]:
# ## https://www.tensorflow.org/guide/mixed_precision ## TF mixed precision - pytorch requires other setup
# from tensorflow.keras.mixed_precision import experimental as mixed_precision

# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)
# ## will need to correct in places, e.g.: 
# ## outputs = layers.Activation('softmax', dtype='float32', name='predictions')(x)

#### Features to add:
* Lag 
* Rank (popularity) of city, country (in general, +- given booker country)
* Count of hotel; user, trip size ? (may be leaky )
* Seasonal features - Holidays? , datetime

Aggregate feats:
* user changed country? last booking (lag 1) country change? 
* max/min/avg popularity rank of previous locations visited



We should create a dictionary of the rank, count, city/country etc' feats, so we can easily merge them when making more "negative" samples/feats for ranking.


* Consider using a df2 of df without dates + drop_duplicates, +- without user/trip id (After calcing that) .


Leaky or potentially leaky (Dependso n test set): 
* Target freq features - frequency of target city, given source county +- affiliate +- month of year +- given country (and interactions of target freq). 
    * Risk of leaks - depends of test data has temporal split or not. 
    * cartboost can do target encode, but this lets us do it for interactions, e.g. target city freq given the 2 countries and affiliate.
    * beware overfitting! 

In [101]:
max_epochs = 1000
FASTRUN = False #True False

if FASTRUN:
    max_epochs = 500

In [6]:
SAVE_OUTPUTS = True ## save intermediate files to disk

TARGET_COL = "label"
# most basic categorical columns , without 'user_id', , 'utrip_id' ordevice_class - used for count encoding/filtering
BASE_CAT_COLS = ['city_id',  'affiliate_id', 'booker_country', 'hotel_country']

### features to get lags for. Not very robust. May want different feats for lags before -1
LAG_FEAT_COLS = ['city_id', 
#                  'device_class',
       'affiliate_id',
#                  'booker_country',
                 'hotel_country', 
       'duration', 'same_country', 
#                  'checkin_weekday',
       'checkin_week',
#         'checkout_weekday',
       'city_id_count', 'affiliate_id_count',
                 'hotel_country_count', 
#                  'checkin_week_count',
                 'city_id_rank_by_hotel_country',
#        'city_id_rank_by_booker_country', ## there are only 5 booker countries
                 'city_id_rank_by_affiliate',
#        'affiliate_id_rank_by_hotel_country',
#        'affiliate_id_rank_by_booker_country', 
                ]

CAT_FEAT_NAMES = ["booker_country", "device_class","affiliate_id",
                  "city_id","hotel_country",
#                   "utrip_id", ## changed
#                   "user_id", ## ? could use lower dim - depends on train/test overlap
                  "checkin_week",#"checkout_week",
#                     "checkin_weekday",
    "lag1_city_id","lag1_hotel_country","lag1_affiliate_id", #"lag1_device_class",
     "lag2_city_id","lag2_hotel_country",#"lag2_affiliate_id",#"lag2_device_class",
       "lag3_city_id","lag3_hotel_country", #"lag3_booker_country","lag3_affiliate_id","lag3_device_class",
                  "first_hotel_country","first_city_id"
                 ]

TRAIN_FILE_PATH = "booking_train_set.csv" #"/content/drive/MyDrive/booking_wisdom/booking_train_set.csv" #"booking_train_set.csv"

LIST_FILE_PATH =   "all_candidates_list.parquet"#"predictions_list_proc_v2.csv.gz"#"predictions_list_proc_v2.csv.gz"

TEST_FILE_PATH = "booking_test_set.csv" #"/content/drive/MyDrive/booking_wisdom/booking_train_set.csv" #"booking_train_set.csv"
# TEST_LIST_FILE_PATH =  "list_booking_train.csv.gz"#"/content/drive/MyDrive/booking_wisdom/list_booking_train.csv.gz" #"list_booking_train.csv.gz"


In [7]:
# https://stackoverflow.com/questions/33907537/groupby-and-lag-all-columns-of-a-dataframe
# https://stackoverflow.com/questions/62924987/lag-multiple-variables-grouped-by-columns
## lag features with groupby over many columns: 
def groupbyLagFeatures(df:pd.DataFrame,lag:[]=[1,2],group="utrip_id",lag_feature_cols=[]):
    """
    lag features with groupby over many columns
    https://stackoverflow.com/questions/62924987/lag-multiple-variables-grouped-by-columns"""
    if len(lag_feature_cols)>0:
        df=pd.concat([df]+[df.groupby(group)[lag_feature_cols].shift(x).add_prefix('lag'+str(x)+"_") for x in lag],axis=1)
    else:
         df=pd.concat([df]+[df.groupby(group).shift(x).add_prefix('lag'+str(x)+"_") for x in lag],axis=1)
    return df

def groupbyFirstLagFeatures(df:pd.DataFrame,group="user_id",lag_feature_cols=[]):
    """
    Get  first/head value lag-like of features with groupby over columns. Assumes sorted data!
    """
    if len(lag_feature_cols)>0:
        df=pd.concat([df]+[df.groupby(group)[lag_feature_cols].transform("first").add_prefix("first_")],axis=1)
    else:
#          df=pd.concat([df]+[df.groupby(group).first().add_prefix("first_")],axis=1)
        df=pd.concat([df]+[df.groupby(group).transform("first").add_prefix("first_")],axis=1)
    return df

######## Get n most popular items, per group
def most_popular(group, n_max=4):
    """Find most popular hotel clusters by destination
    Define a function to get most popular hotels for a destination group.

    Previous version used nlargest() Series method to get indices of largest elements. But the method is rather slow.
    Source: https://www.kaggle.com/dvasyukova/predict-hotel-type-with-pandas
    """
    relevance = group['relevance'].values
    hotel_cluster = group['hotel_cluster'].values
    most_popular = hotel_cluster[np.argsort(relevance)[::-1]][:n_max]
    return np.array_str(most_popular)[1:-1] # remove square brackets

from tensorflow.keras.metrics import top_k_categorical_accuracy
def top_4_accuracy(y_true, y_pred):
    """will only work if doing multiclass predictions"""
    return top_k_categorical_accuracy(y_true, y_pred, k=4)

## https://codereview.stackexchange.com/questions/149306/select-the-n-most-frequent-items-from-a-pandas-groupby-dataframe
# https://stackoverflow.com/questions/52073054/group-by-a-column-to-find-the-most-frequent-value-in-another-column
## can get modes (sorted)
# https://stackoverflow.com/questions/50592762/finding-most-common-values-with-pandas-groupby-and-value-counts
## df.groupby('tag')['category'].agg(lambda x: x.value_counts().index[0])
# https://stackoverflow.com/questions/15222754/groupby-pandas-dataframe-and-select-most-common-value
# source2.groupby(['Country','City'])['Short name'].agg(pd.Series.mode)


# def fit_cbr_model(loss_function, additional_params=None, train_pool=train_pool, test_pool=test_pool,
#                   CV=True,return_model=False):
#     """
#     Fit catboost ranking model, or CV (cross validate) and return scores ; with a given loss function
#     https://colab.research.google.com/github/catboost/tutorials/blob/master/ranking/ranking_tutorial.ipynb#scrollTo=RVNW0nowbtxH
#     >>>model = fit_model('PairLogit', {'custom_metric': ['PrecisionAt:top=10', 'RecallAt:top=4']})
    
#     """
#     parameters = deepcopy(default_parameters)
#     parameters['loss_function'] = loss_function
#     parameters['train_dir'] = loss_function
    
#     if additional_params is not None:
#         parameters.update(additional_params)
         
#     if CV:
#         scores = cv(train_pool,params=parameters, plot=True,fold_count=2)
#         scores = scores[scores.columns[~scores.columns.str.contains("std|pair",case=False)]]
#         display(scores.tail())
        
#     ## train model to get hacky top4 accuracy
#     model = CatBoost(parameters)
#     model.fit(train_pool, eval_set=test_pool, plot=False)        
#     y_test_pred = model.predict(X_test,prediction_type="Probability")[:,1]
#     print("Test:")
#     top4_acc = top4_accuracy(model,y_test_pred)
#     auc_score = roc_auc_score(y_true=y_test,y_score=y_test_pred)
#     print("Roc-AUC (test): {0:.4f}".format(100*auc_score))
 
#     if return_model:
#         return model

In [8]:
dtypes_dict = {
#     "utrip_id":"category",
#     "city_id":"int32",
#     "rank":"int16",
    "label": "int8",#"uint8",
    "affiliate_id":"int32",
#     "device_class":"category",
#     "booker_country":"string", ## category would save more memory but intereferes with feature engineering
#     "hotel_country":"string"
}

In [9]:
# pd.read_csv(TRAIN_FILE_PATH,usecols=["city_id"]).min() ## lowest normal num in train city_id is 2

In [10]:
if FASTRUN:
    df = pd.read_csv(TRAIN_FILE_PATH,
                     nrows=412_280,
                     index_col=[0],
                     parse_dates=["checkin","checkout"],infer_datetime_format=True, dtype=dtypes_dict)
else:
    df = pd.read_csv(TRAIN_FILE_PATH,
                     index_col=[0],
                     parse_dates=["checkin","checkout"],infer_datetime_format=True, dtype=dtypes_dict)

## list of candidate predictions (100 per utrip_id)
df_list = pd.read_parquet(LIST_FILE_PATH,
#                         usecols=[0,2,3], # skip city_id col
#                         dtype=dtypes_dict
                     )

df_list = df_list.loc[df_list["rank"]<50]
print("rank of labels (target =1):")
# print(df_list.loc[df_list["label"]==1]["rank"].describe(percentiles=[.25,.5,.75,.9,.95,.99]))

print(df_list.shape[0])

# assert df_list.drop_duplicates(["utrip_id","city_id"]).shape[0] == df_list.shape[0]

# if FASTRUN:
#     df_list = df_list.loc[df_list["rank"]<=25]
# #     df_list = df_list.loc[(df_list["rank"]<=12) | (df_list["label"]==1)]  
# else:
#     #### downsampole train data, less so for test data/candidates
#     # # ## HACK due to running out of mempory ! sample of data 
    
#     train_ids_all = list(df["utrip_id"].unique())
#     print("len(train_ids_all)",len(train_ids_all))
#     df_list_test = df_list.loc[~df_list["utrip_id"].isin(train_ids_all)]
    
#     ## temp: for faster iterations
#     df_list_test = df_list_test.loc[df_list_test["rank"]<= 100]
    
#     print("list test shape",df_list_test.shape)
#     df_list_train = df_list.loc[df_list["utrip_id"].isin(train_ids_all)]
#     print("train test shape",df_list_train.shape)
    
#     df_list_1 = df_list_train.loc[(df_list_train["rank"]<=60) | (df_list_train["label"]==1)]    
#     df_list_2 = df_list_train.loc[(df_list_train["rank"]>60) & (df_list_train["label"]==0)].groupby(["utrip_id"]).sample(n=15,random_state=42)
#     df_list = pd.concat([df_list_1,df_list_2,df_list_test])
#     del df_list_1,df_list_2,df_list_test,df_list_train
#     gc.collect()

print("df list shape",df_list.shape[0])

### df_test has "missing" cities, hotel countries
 ## replace missing cities with nans or -9
### df_test["city_id"] = df_test["city_id"].replace(0,np.nan)

# df_list = df_list.loc[df_list["city_id"] >0]

df_list.set_index("utrip_id",inplace=True)
                     
df.sort_values(["user_id","checkin"],inplace=True)
# df_list.set_index("utrip_id",inplace=True)


## add feature - score relative to rank (could add vs utrip_id - equiv to rank)
# df_list["score_norm"] = df_list["score"].div(df_list.groupby("rank")["score"].transform("mean"))#.round(3)


display(df)
# display(df_list)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


rank of labels (target =1):
8855028
df list shape 8855028


,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id
1004862,29,2016-07-09,2016-07-11,47054,desktop,1601,Elbonia,Elbonia,29_1
1004863,29,2016-07-11,2016-07-13,34444,desktop,1601,Elbonia,Elbonia,29_1
1004864,29,2016-07-13,2016-07-16,12291,desktop,1601,Elbonia,Elbonia,29_1
1004865,29,2016-07-16,2016-07-18,16386,desktop,8132,Elbonia,Elbonia,29_1
897811,81,2016-05-15,2016-05-16,33665,desktop,9924,Elbonia,Elbonia,81_1
...,...,...,...,...,...,...,...,...,...
1072993,6258065,2016-04-19,2016-04-20,55044,mobile,9452,Gondal,Pullamawang,6258065_1
420479,6258087,2016-08-03,2016-08-04,17754,desktop,2436,Gondal,Gondal,6258087_1
420480,6258087,2016-08-04,2016-08-05,50073,desktop,2436,Gondal,Gondal,6258087_1
420481,6258087,2016-08-05,2016-08-06,11662,desktop,2436,Gondal,Gondal,6258087_1


In [11]:
df_list.loc[df_list["label"]==1]

,city_id,score,label,rank,score_norm
utrip_id,,,,,
1000027_1,30628,4.025222,1.0,23,0.709
1000136_1,32627,5.456468,1.0,30,1.002
1000208_1,56872,7.744469,1.0,2,1.053
1000241_1,28319,4.717289,1.0,44,0.925
1000255_1,40502,7.905730,1.0,8,1.212
...,...,...,...,...,...
999341_2,9699,4.109669,1.0,1,0.539
999385_2,8344,7.451572,1.0,5,1.088
999583_3,47700,6.152070,1.0,4,0.881


In [12]:
df_test = pd.read_csv(TEST_FILE_PATH, parse_dates=["checkin","checkout"],infer_datetime_format=True, dtype=dtypes_dict)
### note the missing city id in last rows
print(df_test.shape)
test_num_ids = df_test["utrip_id"].nunique()
print("test_num_ids",test_num_ids)
df_test["train"] = 0

### df_test has "missing" cities, hotel countries
# df_list["city_id"] = df_list["city_id"].replace(0,-9)# np.nan ## replace missing cities with nans or -9
df_test["city_id"] = df_test["city_id"].replace(0,np.nan) ## 0 are missing cities here

df_test

(378667, 11)
test_num_ids 70662


,user_id,checkin,checkout,device_class,affiliate_id,booker_country,utrip_id,row_num,total_rows,city_id,hotel_country,train
0,1000066,2016-07-21,2016-07-23,desktop,9924,Gondal,1000066_2,1,4,56430.0,Urkesh,0
1,1000066,2016-07-23,2016-07-25,desktop,9924,Gondal,1000066_2,2,4,41971.0,Urkesh,0
2,1000066,2016-07-25,2016-07-28,desktop,9924,Gondal,1000066_2,3,4,5797.0,Urkesh,0
3,1000066,2016-07-28,2016-07-31,mobile,2436,Gondal,1000066_2,4,4,NaN,NaN,0
4,1000270,2016-02-08,2016-02-09,mobile,9452,The Devilfire Empire,1000270_1,1,4,50075.0,The Devilfire Empire,0
...,...,...,...,...,...,...,...,...,...,...,...,...
378662,999911,2016-10-07,2016-10-08,desktop,9598,Gondal,999911_1,5,5,NaN,NaN,0
378663,999991,2016-08-15,2016-08-17,desktop,8065,Elbonia,999991_3,1,4,29770.0,Elbonia,0
378664,999991,2016-08-18,2016-08-19,desktop,8065,Elbonia,999991_3,2,4,36170.0,Carpathia,0
378665,999991,2016-08-19,2016-08-20,tablet,3631,Elbonia,999991_3,3,4,52155.0,Elbonia,0


In [13]:
### add features to be consistent with test set of row in trip, and total trips in trip
df["row_num"] = df.groupby("utrip_id")["checkin"].rank(ascending=True,pct=False).astype(int)
utrip_counts = df["utrip_id"].value_counts()
df["total_rows"] = df["utrip_id"].map(utrip_counts)

df["train"] = 1

df["total_rows"].describe()

count    1.166835e+06
mean     6.117143e+00
std      2.796383e+00
min      1.000000e+00
25%      4.000000e+00
50%      5.000000e+00
75%      7.000000e+00
max      4.800000e+01
Name: total_rows, dtype: float64

In [14]:
### stack train, test for now (easier feat eng)
df = pd.concat([df,df_test])
print(df.shape[0])

df["last"] = (df["row_num"] ==df["total_rows"]).astype(int)
assert 0 == df[["row_num","total_rows","last"]].isna().sum().max()
# df[["row_num","total_rows","last"]].describe()

1545502


In [15]:
### Drop duplicate, consecutive rows
df = df.loc[(df[["city_id","utrip_id","booker_country","last"]].shift() != df[["city_id","utrip_id","booker_country","last"]]).max(axis=1)]
print(df.shape[0])

## add alt total rows as a diff, for marking cases where we dropped dupes.

utrip_counts_2 = df["utrip_id"].value_counts()
df["diff_total_rows"] = df["utrip_id"].map(utrip_counts_2)
df["diff_total_rows"] =df["total_rows"] -  df["diff_total_rows"]

df[["diff_total_rows","total_rows"]].describe()

1461147


,diff_total_rows,total_rows
count,1.461147e+06,1.461147e+06
mean,2.287990e-01,6.095443e+00
std,6.714189e-01,2.716051e+00
min,0.000000e+00,1.000000e+00
25%,0.000000e+00,4.000000e+00
50%,0.000000e+00,5.000000e+00
75%,0.000000e+00,7.000000e+00
max,4.200000e+01,4.800000e+01


In [16]:
### todo : add "time since start (min date in data / "duration")"  
### todo : "time since start of trip (min date per trip "duration")"  
df["duration"] = ((df["checkout"] - df["checkin"]).dt.days).astype(int)
df["same_country"] = (df["booker_country"]==df["hotel_country"]).astype(int)

df["checkin_day"] = df["checkin"].dt.day
df["checkin_weekday"] = df["checkin"].dt.weekday
df["checkin_week"] = df["checkin"].dt.isocalendar().week.astype(int) ## week of year
df["checkin_month"] = df["checkin"].dt.month
df["checkin_year"] = df["checkin"].dt.year-2015
df["checkin_day_of_year"] = df["checkin"].dt.dayofyear  # day of year - may overfit , but will allow learning holidays and other checkins on that date 

# df["checkin_quarter"] = df["checkin"].dt.quarter # relatively redundant but may be used for "id"

# df["checkout_weekday"] = df["checkout"].dt.weekday
df["checkout_week"] = df["checkout"].dt.isocalendar().week.astype(int) ## week of year
df["checkout_day"] = df["checkout"].dt.day ## day of month

## cyclical datetime embeddings
## drop originakl variables? 
## TODO:L add for other variables, +- those that we'll embed (week?)

# df['checkin_weekday_sin'] = np.sin(df["checkin_weekday"]*(2.*np.pi/7))
# df['checkin_weekday_cos'] = np.cos(df["checkin_weekday"]*(2.*np.pi/7))

# df['checkin_month_sin'] = np.sin((df["checkin_month"]-1)*(2.*np.pi/12))
# df['checkin_month_cos'] = np.cos((df["checkin_month"]-1)*(2.*np.pi/12))

df['checkin_week_sin'] = np.sin((df["checkin_week"]-1)*(2.*np.pi/53))
df['checkin_week_cos'] = np.cos((df["checkin_week"]-1)*(2.*np.pi/53))

df.drop(["checkin_month"],axis=1,errors="ignore")

### time since first date in dataset (i.,e index proxy)
min_date = df["checkin"].min()
df["time_elapsed_global"] = (df["checkin"] - min_date).dt.days.astype(int)

# #############
# # last number in utrip id - probably which trip number it is:
# df["utrip_number"] = df["utrip_id"].str.split("_",expand=True)[1].astype(int)

### encode string columns - must be consistent with test data 
### IF we can concat test with train, we can just do a single transformation  for the NON TARGET cols
# obj_cols_list = df.select_dtypes("O").columns.values
obj_cols_list = ['device_class','booker_country','hotel_country'] # we could also define when loading data, dtype

for c in obj_cols_list:
    df[c] = df[c].astype("category")
    df[c] = df[c].cat.codes.astype(int)


In [17]:
def map_rare_feats(df,col,thresh=2,replace_val=""):
    ### replace rare variables (under 2 occurrences) with "-1" dummy
    affiliates_counts = df[col].value_counts()
#     print("before:", affiliates_counts)
    print("uniques",df[col].nunique())
    affiliates_counts = affiliates_counts.to_dict()
    # df[col] = df[col].where(df[col].apply(lambda x: x.map(x.value_counts()))>=3, -1)
    df[col] = df[col].where(df[col].map(affiliates_counts)>thresh, replace_val)
    print("afterwards uniques",df[col].nunique())
#     df[col] = df[col].astype(int)
    return(df[col])

In [18]:
### replace rare variables (under 2 occurrences) with "-1" dummy
affiliates_counts = df["affiliate_id"].value_counts()
print("before:", affiliates_counts)
print("uniques",df["affiliate_id"].nunique())
affiliates_counts = affiliates_counts.to_dict()
# df["affiliate_id"] = df["affiliate_id"].where(df["affiliate_id"].apply(lambda x: x.map(x.value_counts()))>=3, -1)
df["affiliate_id"] = df["affiliate_id"].where(df["affiliate_id"].map(affiliates_counts)>4, -1)
df["affiliate_id"] = df["affiliate_id"].astype(int)

# print("after\n",df["affiliate_id"].value_counts())
print("affiliate_id nunique",df["affiliate_id"].nunique())

before: 9924     351030
359      203653
384      113307
9452     101627
4541      54050
          ...  
4798          1
1722          1
10469         1
225           1
2047          1
Name: affiliate_id, Length: 3575, dtype: int64
uniques 3575
affiliate_id nunique 1829


In [19]:
df.city_id.min()

2.0

In [20]:
### duplicate of city, country id columns, as number (may be redundant - we already have count encoding ! )
### replace rare variables (under 2 occurrences) with "-1" dummy
## city id:
affiliates_counts = df["city_id"].value_counts()
# print("before:", affiliates_counts)
print("uniques",df["city_id"].nunique())
affiliates_counts = affiliates_counts.to_dict()
df["numeric_city_id"] = df["city_id"].where(df["city_id"].map(affiliates_counts)>3, -1)
df["numeric_city_id"] = df["numeric_city_id"].astype(int)

## hotel_country: (+ordinal encoding)
affiliates_counts = df["hotel_country"].value_counts()
# print("before:", affiliates_counts)
print("uniques",df["hotel_country"].nunique())
affiliates_counts = affiliates_counts.to_dict()
df["numeric_hotel_country"] = df["hotel_country"].where(df["hotel_country"].map(affiliates_counts)>3, "")
df["numeric_hotel_country"] = df["numeric_hotel_country"].astype("category").cat.codes.astype(int)

uniques 39901
uniques 196


* Add first country, city visited in a trip. 
* Drop first row of a trip

In [21]:
## add the "first" place visited/values
### nopte - will need to drop first row in trip, or impute nans when using this feature 

### first by user results in too much sparsity/rareness for our IDs purposes
df = groupbyFirstLagFeatures(df,group="utrip_id",lag_feature_cols=["hotel_country","city_id","duration","same_country",
                                                                   "affiliate_id", "checkin"]) 
## + first checkin date -> use for time delta feature
# df = df.loc[df["row_num"]>1] ## can't do yet, needed for lag features
print(df[["first_hotel_country","hotel_country","city_id","first_same_country", "first_affiliate_id"]].nunique())
df

first_hotel_country      177
hotel_country            196
city_id                39901
first_same_country         2
first_affiliate_id      1758
dtype: int64


,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id,row_num,total_rows,train,last,diff_total_rows,duration,same_country,checkin_day,checkin_weekday,checkin_week,checkin_month,checkin_year,checkin_day_of_year,checkout_week,checkout_day,checkin_week_sin,checkin_week_cos,time_elapsed_global,numeric_city_id,numeric_hotel_country,first_hotel_country,first_city_id,first_duration,first_same_country,first_affiliate_id,first_checkin
1004862,29,2016-07-09,2016-07-11,47054.0,0,1601,1,46,29_1,1,4,1,0,0,2,1,9,5,27,7,1,191,28,11,0.059241,-0.998244,191,47054,44,46,47054.0,2,1,1601,2016-07-09
1004863,29,2016-07-11,2016-07-13,34444.0,0,1601,1,46,29_1,2,4,1,0,0,2,1,11,0,28,7,1,193,28,13,-0.059241,-0.998244,193,34444,44,46,47054.0,2,1,1601,2016-07-09
1004864,29,2016-07-13,2016-07-16,12291.0,0,1601,1,46,29_1,3,4,1,0,0,3,1,13,2,28,7,1,195,28,16,-0.059241,-0.998244,195,12291,44,46,47054.0,2,1,1601,2016-07-09
1004865,29,2016-07-16,2016-07-18,16386.0,0,8132,1,46,29_1,4,4,1,1,0,2,1,16,5,28,7,1,198,29,18,-0.059241,-0.998244,198,16386,44,46,47054.0,2,1,1601,2016-07-09
897811,81,2016-05-15,2016-05-16,33665.0,0,9924,1,46,81_1,1,4,1,0,0,1,1,15,6,19,5,1,136,20,16,0.845596,-0.533823,136,33665,44,46,33665.0,1,1,9924,2016-05-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378662,999911,2016-10-07,2016-10-08,NaN,0,9598,2,-1,999911_1,5,5,0,1,0,1,0,7,4,40,10,1,281,40,8,-0.996050,-0.088796,281,-1,0,36,5429.0,1,0,9598,2016-10-01
378663,999991,2016-08-15,2016-08-17,29770.0,0,8065,1,46,999991_3,1,4,0,0,0,2,1,15,0,33,8,1,228,33,17,-0.606800,-0.794854,228,29770,44,46,29770.0,2,1,8065,2016-08-15
378664,999991,2016-08-18,2016-08-19,36170.0,0,8065,1,32,999991_3,2,4,0,0,0,1,0,18,3,33,8,1,231,33,19,-0.606800,-0.794854,231,36170,30,46,29770.0,2,1,8065,2016-08-15
378665,999991,2016-08-19,2016-08-20,52155.0,2,3631,1,46,999991_3,3,4,0,0,0,1,1,19,4,33,8,1,232,33,20,-0.606800,-0.794854,232,52155,44,46,29770.0,2,1,8065,2016-08-15


Add relative date/duration feature(s)
* Time since start of trip (first_checkin)

In [22]:
df["total_trip_duration"] = ((df["checkin"] -df["first_checkin"]).dt.days).astype(int)

df["first_checkin_week"] = df["first_checkin"].dt.isocalendar().week.astype(int)
# df[["total_trip_duration","first_checkin","checkin"]]

In [23]:
### change type of date cols to more learnable format

df["checkin"] = (df["checkin"]-d1).dt.days.astype(int16)
df["checkout"] = (df["checkout"]-d1).dt.days.astype(int16)
df["first_checkin"] = (df["first_checkin"]-d1).dt.days.astype(int16)
## # (df["checkin"]-d1).astype('int64')//1e9

In [24]:
##### Following aggregation features - would be best to use time window (sort data) to generate, otherwise they will LEAK! (e.g. nunique countries visited)

### count features (can also later add rank inside groups).
### Some may be leaks (# visits in a trip should use time window?) , and do users repeat? 
### can add more counts of group X time period (e.g. affiliate X month of year)
## alt way to get counts/freq :
# freq = df["city_id"].value_counts()
# df["city_id_count"] = df["city_id"].map(freq)
# print(df["city_id_count"].describe())

count_cols = [ 'city_id','affiliate_id',  'hotel_country', 
             'user_id', 
#              "checkin_month",
              "checkin_week",
              "checkin_day_of_year"
             ]

for c in count_cols:
    df[f"{c}_count"] = df.groupby([c])["duration"].transform("size")
    
########################################################
## nunique per trip
### https://stackoverflow.com/questions/46470743/how-to-efficiently-compute-a-rolling-unique-count-in-a-pandas-time-series

nunique_cols = [ 'city_id', 'hotel_country','affiliate_id', 
                'booker_country',"duration","checkin_weekday"]
# df["nunique_booker_countries"] = df.groupby("utrip_id")["booker_country"].nunique()
# df["nunique_hotel_country"] = df.groupby("utrip_id")["hotel_country"].nunique()
for c in nunique_cols:
    df[f"{c}_nunique"] = df.groupby(["utrip_id"])[c].transform("nunique")

### trips taken by user
df["user_nunique_trips"] = df.groupby("user_id")["utrip_id"].transform("nunique")
print(df.nunique())

########################################################
## get frequency/count feature's rank within a group - e.g. within a country (or affiliate) 
## add "_count" to column name to get count col name, then add rank col 

### ALT/ duplicate feat - add percent rank (instead or in addition)

rank_cols = ['city_id','affiliate_id', 'hotel_country',  # 'booker_country',
#  "checkin_month",
             "checkin_week"
            ]
### what is meaning of groupby and rank of smae variable by same var? Surely should be 1 / unary? 
for c in rank_cols:
    df[f"{c}_rank_by_hotel_country"] = df.groupby(['hotel_country'])[f"{c}_count"].transform("rank")
    df[f"{c}_rank_by_affiliate"] = df.groupby(['affiliate_id'])[f"{c}_count"].transform("rank")
    ### rank by month: potentially strong feature, but must avoid leaks + must be joined back by month
#     df[f"{c}_rank_by_month"] = df.groupby(['checkin_month'])[f"{c}_count"].transform("rank") ## "city_id_rank_by_month" may leak
## there are just 5 booker countries.. 
df["city_id_rank_by_booker_country"] = df.groupby(['booker_country'])[f"city_id_count"].transform("rank")
    
df

user_id                      259159
checkin                         425
checkout                        425
city_id                       39901
device_class                      3
affiliate_id                   1829
booker_country                    5
hotel_country                   196
utrip_id                     288348
row_num                          46
total_rows                       42
train                             2
last                              2
diff_total_rows                  29
duration                         30
same_country                      2
checkin_day                      31
checkin_weekday                   7
checkin_week                     53
checkin_month                    12
checkin_year                      3
checkin_day_of_year             366
checkout_week                    53
checkout_day                     31
checkin_week_sin                 53
checkin_week_cos                 45
time_elapsed_global             425
numeric_city_id             

,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id,row_num,total_rows,train,last,diff_total_rows,duration,same_country,checkin_day,checkin_weekday,checkin_week,checkin_month,checkin_year,checkin_day_of_year,checkout_week,checkout_day,checkin_week_sin,checkin_week_cos,time_elapsed_global,numeric_city_id,numeric_hotel_country,first_hotel_country,first_city_id,first_duration,first_same_country,first_affiliate_id,first_checkin,total_trip_duration,first_checkin_week,city_id_count,affiliate_id_count,hotel_country_count,user_id_count,checkin_week_count,checkin_day_of_year_count,city_id_nunique,hotel_country_nunique,affiliate_id_nunique,booker_country_nunique,duration_nunique,checkin_weekday_nunique,user_nunique_trips,city_id_rank_by_hotel_country,city_id_rank_by_affiliate,affiliate_id_rank_by_hotel_country,affiliate_id_rank_by_affiliate,hotel_country_rank_by_hotel_country,hotel_country_rank_by_affiliate,checkin_week_rank_by_hotel_country,checkin_week_rank_by_affiliate,city_id_rank_by_booker_country
1004862,29,192,194,47054.0,0,1601,1,46,29_1,1,4,1,0,0,2,1,9,5,27,7,1,191,28,11,0.059241,-0.998244,191,47054,44,46,47054.0,2,1,1601,192,0,27,136.0,6287,64177,4,35167,5526,4,1,2,1,2,3,1,24797.5,2498.5,14027.0,3144.0,32089.0,3331.0,29350.0,2734.0,110785.0
1004863,29,194,196,34444.0,0,1601,1,46,29_1,2,4,1,0,0,2,1,11,0,28,7,1,193,28,13,-0.059241,-0.998244,193,34444,44,46,47054.0,2,1,1601,192,2,27,43.0,6287,64177,4,47360,6167,4,1,2,1,2,3,1,18047.5,1492.0,14027.0,3144.0,32089.0,3331.0,43351.5,4427.0,69372.0
1004864,29,196,199,12291.0,0,1601,1,46,29_1,3,4,1,0,0,3,1,13,2,28,7,1,195,28,16,-0.059241,-0.998244,195,12291,44,46,47054.0,2,1,1601,192,4,27,38.0,6287,64177,4,47360,6475,4,1,2,1,2,3,1,17303.5,1407.5,14027.0,3144.0,32089.0,3331.0,43351.5,4427.0,65297.5
1004865,29,199,201,16386.0,0,8132,1,46,29_1,4,4,1,1,0,2,1,16,5,28,7,1,198,29,18,-0.059241,-0.998244,198,16386,44,46,47054.0,2,1,1601,192,7,27,4.0,28591,64177,4,47360,7255,4,1,2,1,2,3,1,3708.5,1500.5,23692.5,14296.0,32089.0,15566.5,43351.5,21471.5,12633.0
897811,81,137,138,33665.0,0,9924,1,46,81_1,1,4,1,0,0,1,1,15,6,19,5,1,136,20,16,0.845596,-0.533823,136,33665,44,46,33665.0,1,1,9924,137,0,19,29.0,351030,64177,4,26519,3773,4,2,1,1,2,4,1,15582.0,44550.5,55234.0,175515.5,32089.0,203526.0,21018.5,143682.5,57157.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378662,999911,282,283,NaN,0,9598,2,-1,999911_1,5,5,0,1,0,1,0,7,4,40,10,1,281,40,8,-0.996050,-0.088796,281,-1,0,36,5429.0,1,0,9598,276,6,39,NaN,31096,70662,5,41637,5360,4,2,4,1,2,5,1,NaN,NaN,22857.5,15548.5,35331.5,19613.5,43080.5,17809.5,NaN
378663,999991,229,231,29770.0,0,8065,1,46,999991_3,1,4,0,0,0,2,1,15,0,33,8,1,228,33,17,-0.606800,-0.794854,228,29770,44,46,29770.0,2,1,8065,229,0,33,7028.0,4705,64177,4,66496,10571,3,3,2,1,2,4,1,60663.5,4272.0,12478.0,2353.0,32089.0,2684.0,55143.0,4270.5,273972.5
378664,999991,232,233,36170.0,0,8065,1,32,999991_3,2,4,0,0,0,1,0,18,3,33,8,1,231,33,19,-0.606800,-0.794854,231,36170,30,46,29770.0,2,1,8065,229,3,33,2835.0,4705,25990,4,66496,9562,3,3,2,1,2,4,1,21728.0,3820.0,5913.5,2353.0,12995.5,1449.5,22044.0,4270.5,248980.0
378665,999991,233,234,52155.0,2,3631,1,46,999991_3,3,4,0,0,0,1,1,19,4,33,8,1,232,33,20,-0.606800,-0.794854,232,52155,44,46,29770.0,2,1,8065,229,4,33,9.0,32303,64177,4,66496,9018,3,3,2,1,2,4,1,8010.0,1808.0,26660.5,16152.0,32089.0,18150.5,55143.0,28280.0,26995.5


#### DF of features per city
* We can join this to the list of candidate cities

WARNING! Some features here are NOT Calculate d correctly - we see too many values, they are not unique per city_id & hotel!! 

In [25]:
# CITY_FEATURES = df.columns[(df.columns.str.contains("city|hotel",case=False)) & (~df.columns.str.contains("first_|month",case=False))]
CITY_FEATURES = ['city_id', 'hotel_country', 'city_id_count', 'hotel_country_count',
       'city_id_rank_by_hotel_country',
#                  "city_id_rank_by_month" ## can't be added like this, depends on the month for joining!
                "numeric_hotel_country","numeric_city_id"]

print("city features\n",CITY_FEATURES)

df_city_features = df[CITY_FEATURES].drop_duplicates().set_index("city_id") 
## duplicate "city_id","hotel_country" , and keep them as numeric columns. Note, this may overlap with the count features, may be redundnat! 

# print(df["city_id"].nunique(dropna=False))
# print(df_city_features.shape)

# assert df["city_id"].nunique(dropna=False) == df_city_features.shape[0]


## drop the num city/country cols, not needed after, will join them from df_city_features
df.drop(["numeric_hotel_country","numeric_city_id"],axis=1,errors="ignore",inplace=True)

city features
 ['city_id', 'hotel_country', 'city_id_count', 'hotel_country_count', 'city_id_rank_by_hotel_country', 'numeric_hotel_country', 'numeric_city_id']


In [26]:
df.loc[df["city_id_count"]>=10]["city_id"].nunique()

10991

In [27]:
# assert df.isna().sum().max() ==0

In [28]:
## check coverage of top k classes
y_last = df.loc[(df["last"]==1) & (df["train"]==1)]["city_id"].dropna().astype(int)
print("sum top4 total percentage:",100*y_last.value_counts(normalize=True)[0:4].sum().round(3)) ##5.7%
print("sum top200 total percentage:",100*y_last.value_counts(normalize=True)[0:200].sum().round(3)) ## 48%
print("sum top500 total percentage:",100*y_last.value_counts(normalize=True)[0:500].sum().round(3)) ### 61%
print("sum top1000 total percentage:",100*y_last.value_counts(normalize=True)[0:1000].sum().round(3)) ### 71 %

sum top4 total percentage: 5.7
sum top200 total percentage: 48.5
sum top500 total percentage: 61.8
sum top1000 total percentage: 71.39999999999999


### add lag features + Train/test/data split
* Lag feats (remember for categorical)
* Drop leak features (target values - country, city)

* drop instances  that lack history (e.g. at least 3d step and onwards) - by dropna in lag feat
* fill nans
* Split train/test by `user id` / split could maybe be by `utrip ID` ? ? 
    * Test - only last trip
    *  stratified train/test split by class - then drop any train rows with overlap with tests' IDs.  
        * Could also stratify by users, but risks some classes being non present in test

* TODO: aggregate features o ntime series - e.g. mean, max, min duration
    * We already have nunique
###### WE may not need to drop these features anymore - but we may want to join them by city id
*We  join city id features after adding the list of caNDIDATes

In [29]:
### features to drop - not usable, or leaks (e.g. aggregations on target) , or hotel/city specific (should be added from per city features)

DROP_FEATS = [ #'user_id', ## drop seperately
#     'checkin',  # is ok if turned into number
#     'checkout',
              'hotel_country',
              'city_id_count','same_country',
#               'utrip_id',
             'city_id_count','hotel_country_count',
              'city_id_nunique', 'hotel_country_nunique',
              'city_id_rank_by_hotel_country','city_id_rank_by_booker_country', 'city_id_rank_by_affiliate',
              'affiliate_id_rank_by_hotel_country','affiliate_id_rank_by_booker_country', 'affiliate_id_rank_by_affiliate',
              'hotel_country_rank_by_hotel_country',
       'hotel_country_rank_by_booker_country','hotel_country_rank_by_affiliate',
              'booker_country_rank_by_hotel_country',
              'booker_country_rank_by_booker_country',
              'checkin_month_rank_by_hotel_country',
             ]

# df2.drop(DROP_FEATS,axis=1).columns

In [30]:
print("df.shape",df.shape)
# ### lag features - last n visits
df_feat = groupbyLagFeatures(df=df.copy(), 
                   lag=[1,2],group="utrip_id",lag_feature_cols=LAG_FEAT_COLS)
# df_feat = df_feat.dropna(subset=["lag2_city_id"]).sample(frac=1)
df_feat = df_feat.loc[df_feat["last"]==1].drop(["last","row_num"],axis=1,errors="ignore")
# df_feat = df_feat.drop(DROP_FEATS,axis=1,errors="ignore")
print("df_feat",df_feat.shape)
assert df_feat["utrip_id"].nunique() == df["utrip_id"].nunique()

df.shape (1461147, 57)
df_feat (288348, 77)


In [31]:
MINI_LAG_FEAT_COLS = ["city_id_count","city_id","hotel_country"] # ,"duration"

df_more_lags = groupbyLagFeatures(df=df.copy(), 
                   lag=[3],group="utrip_id",lag_feature_cols=MINI_LAG_FEAT_COLS)#[MINI_LAG_FEAT_COLS+["utrip_id","last"]]
lag_cols = [x for x in df_more_lags.columns if x.startswith("lag")]
df_more_lags = df_more_lags.loc[df_more_lags["last"]==1].drop(["last"],axis=1,errors="ignore")
df_more_lags = df_more_lags[lag_cols+["utrip_id"]]
df_more_lags

,lag3_city_id_count,lag3_city_id,lag3_hotel_country,utrip_id
1004865,136.0,47054.0,46.0,29_1
897814,29.0,33665.0,46.0,81_1
770550,155.0,38793.0,166.0,136_2
117283,213.0,55990.0,122.0,136_4
117413,NaN,NaN,NaN,149_1
...,...,...,...,...
378643,306.0,40643.0,112.0,999674_1
378650,62.0,36135.0,60.0,999797_1
378655,29.0,12983.0,76.0,999862_1
378662,1794.0,21987.0,36.0,999911_1


In [32]:
df_feat = df_feat.merge(df_more_lags,on="utrip_id",how="left")
del df_more_lags
print("df_feat",df_feat.shape)
assert df_feat["utrip_id"].nunique() == df["utrip_id"].nunique()

df_feat (288348, 80)


In [33]:
del df
gc.collect()

126808

In [34]:
### impute missing  values - for categoricals at least - strings, numbers
OBJECT_COLS = list(df_feat.select_dtypes("O").columns)
df_feat[OBJECT_COLS] = df_feat[OBJECT_COLS].fillna("")
df_feat[CAT_FEAT_NAMES] = df_feat[CAT_FEAT_NAMES].fillna(-1)
## maybe impute numerics? (for integer type, less data usage)

df_feat[CAT_FEAT_NAMES] = df_feat[CAT_FEAT_NAMES].apply(pd.to_numeric, errors='ignore',downcast="integer")

# NUMERIC_COLS = [item for item in list(df_feat.columns.drop(TARGET_COL))  if item not in CAT_FEAT_NAMES ]
NUMERIC_COLS = [item for item in list(df_feat.select_dtypes("number").columns)  if item not in CAT_FEAT_NAMES ]

print(len(NUMERIC_COLS))
print("numeric cols",NUMERIC_COLS)

# df_feat[CAT_FEAT_NAMES] = df_feat[CAT_FEAT_NAMES].apply(pd.to_numeric, errors='ignore',downcast="integer")

# df_feat[CAT_FEAT_NAMES] = df_feat[CAT_FEAT_NAMES].astype("category")


## downcast dtypes
df_feat = df_feat.apply(pd.to_numeric, errors='ignore',downcast="float").apply(pd.to_numeric, errors='ignore',downcast="integer")

df_feat[OBJECT_COLS+CAT_FEAT_NAMES] = df_feat[OBJECT_COLS+CAT_FEAT_NAMES].astype("category")

# df_feat.sort_values(["user_id","utrip_steps_from_end"])
print(df_feat.info(memory_usage="deep"))
display(df_feat)

64
numeric cols ['user_id', 'checkin', 'checkout', 'total_rows', 'train', 'diff_total_rows', 'duration', 'same_country', 'checkin_day', 'checkin_weekday', 'checkin_month', 'checkin_year', 'checkin_day_of_year', 'checkout_week', 'checkout_day', 'checkin_week_sin', 'checkin_week_cos', 'time_elapsed_global', 'first_duration', 'first_same_country', 'first_affiliate_id', 'first_checkin', 'total_trip_duration', 'first_checkin_week', 'city_id_count', 'affiliate_id_count', 'hotel_country_count', 'user_id_count', 'checkin_week_count', 'checkin_day_of_year_count', 'city_id_nunique', 'hotel_country_nunique', 'affiliate_id_nunique', 'booker_country_nunique', 'duration_nunique', 'checkin_weekday_nunique', 'user_nunique_trips', 'city_id_rank_by_hotel_country', 'city_id_rank_by_affiliate', 'affiliate_id_rank_by_hotel_country', 'affiliate_id_rank_by_affiliate', 'hotel_country_rank_by_hotel_country', 'hotel_country_rank_by_affiliate', 'checkin_week_rank_by_hotel_country', 'checkin_week_rank_by_affiliat

,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id,total_rows,train,diff_total_rows,duration,same_country,checkin_day,checkin_weekday,checkin_week,checkin_month,checkin_year,checkin_day_of_year,checkout_week,checkout_day,checkin_week_sin,checkin_week_cos,time_elapsed_global,first_hotel_country,first_city_id,first_duration,first_same_country,first_affiliate_id,first_checkin,total_trip_duration,first_checkin_week,city_id_count,affiliate_id_count,hotel_country_count,user_id_count,checkin_week_count,checkin_day_of_year_count,city_id_nunique,hotel_country_nunique,affiliate_id_nunique,booker_country_nunique,duration_nunique,checkin_weekday_nunique,user_nunique_trips,city_id_rank_by_hotel_country,city_id_rank_by_affiliate,affiliate_id_rank_by_hotel_country,affiliate_id_rank_by_affiliate,hotel_country_rank_by_hotel_country,hotel_country_rank_by_affiliate,checkin_week_rank_by_hotel_country,checkin_week_rank_by_affiliate,city_id_rank_by_booker_country,lag1_city_id,lag1_affiliate_id,lag1_hotel_country,lag1_duration,lag1_same_country,lag1_checkin_week,lag1_city_id_count,lag1_affiliate_id_count,lag1_hotel_country_count,lag1_city_id_rank_by_hotel_country,lag1_city_id_rank_by_affiliate,lag2_city_id,lag2_affiliate_id,lag2_hotel_country,lag2_duration,lag2_same_country,lag2_checkin_week,lag2_city_id_count,lag2_affiliate_id_count,lag2_hotel_country_count,lag2_city_id_rank_by_hotel_country,lag2_city_id_rank_by_affiliate,lag3_city_id_count,lag3_city_id,lag3_hotel_country
0,29,199,201,16386,0,8132,1,46,29_1,4,1,0,2,1,16,5,28,7,1,198,29,18,-0.059241,-0.998244,198,46,47054,2,1,1601,192,7,27,4.0,28591,64177,4,47360,7255,4,1,2,1,2,3,1,3708.5,1500.5,23692.5,14296.0,32089.0,15566.5,43351.5,21471.5,12633.0,12291,1601,46,3.0,1.0,28.0,38.0,6287.0,64177.0,17303.5,1407.5,34444,1601.0,46,2.0,1.0,28.0,43.0,6287.0,64177.0,18047.5,1492.0,136.0,47054,46
1,81,141,143,19335,0,9924,1,90,81_1,4,1,0,2,0,19,3,20,5,1,140,20,21,0.776524,-0.630088,140,46,33665,1,1,9924,137,4,19,812.0,351030,24424,4,27532,3974,4,2,1,1,2,4,1,13678.5,184583.0,21388.5,175515.5,12212.5,95264.5,5155.5,162467.5,193248.0,22065,9924,90,1.0,0.0,20.0,2279.0,351030.0,24424.0,17718.0,250526.0,15506,9924.0,90,2.0,0.0,20.0,16.0,351030.0,24424.0,1576.5,31039.5,29.0,33665,46
2,136,121,123,45188,0,9924,4,166,136_2,4,1,0,2,0,29,4,17,4,1,120,17,1,0.947326,-0.320270,120,166,38793,1,0,9924,118,3,17,3014.0,351030,9116,11,23183,3187,4,2,1,1,2,4,2,7609.5,269873.5,7899.0,175515.5,4558.5,43290.0,2521.5,101758.0,231833.0,60274,9924,157,1.0,0.0,17.0,2002.0,351030.0,5539.0,4538.5,246943.0,28545,9924.0,157,1.0,0.0,17.0,964.0,351030.0,5539.0,3055.5,195215.0,155.0,38793,166
3,136,273,274,41113,0,9924,4,122,136_4,7,1,0,1,0,28,2,39,9,1,272,39,29,-0.978557,-0.205979,272,122,52933,2,0,9924,265,8,38,378.0,351030,23931,11,46892,6546,7,1,1,1,2,6,2,11023.5,138731.5,20309.5,175515.5,11966.0,88606.5,16570.5,244267.5,79923.0,45399,9924,122,1.0,0.0,39.0,178.0,351030.0,23931.0,7415.5,103217.0,46411,9924.0,122,1.0,0.0,39.0,1227.0,351030.0,23931.0,17195.0,210605.5,213.0,55990,122
4,149,405,407,28733,0,5583,4,79,149_1,4,1,2,2,0,7,1,6,2,2,38,6,9,0.558647,0.829406,404,79,10485,2,0,3417,401,4,5,639.0,380,50256,2,21321,2786,2,1,2,1,1,2,1,12988.0,94.0,2267.0,190.5,25128.5,203.0,25656.5,115.0,103088.5,10485,3417,79,2.0,0.0,5.0,5600.0,4154.0,50256.0,35343.5,3233.0,-1,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288343,999674,42,46,-1,0,9924,2,-1,999674_1,6,0,0,4,0,10,2,6,2,1,41,6,14,0.558647,0.829406,41,23,657,1,0,9627,35,7,5,NaN,351030,70662,11,21321,2950,5,3,3,1,3,5,2,NaN,NaN,62389.5,175515.5,35331.5,220742.5,17649.0,85182.0,NaN,41237,9924,112,1.0,0.0,6.0,201.0,351030.0,2265

Save output (with multiclass target column)

In [35]:
# if SAVE_OUTPUTS:
#     df_feat.loc[df_feat["train"]==0].drop(DROP_FEATS+["train","city_id"],axis=1,errors="ignore").to_parquet(path="df_feat_candidates_multiclass.parquet",index=False)
#     df_feat.loc[df_feat["train"]==1].drop(DROP_FEATS+["train"],axis=1,errors="ignore").to_parquet(path="df_feat_train_multiclass.parquet",index=False)

### join with predicted candidates
* Note: be careful - features for the city/item must match the "new" city as well!! e.g. city rank

* Also, we may want to keep the original city id for cases where positive wasn't in candidates? 


In [36]:
### new in train+test prediction notebook
df_feat.drop(["city_id","hotel_country"],axis=1,errors="ignore",inplace=True)

In [37]:
# df_feat =df_list.drop(["city_id"],axis=1,errors="ignore").merge(df_feat,how="inner",on="utrip_id")
df_feat =df_list.merge(df_feat,how="inner",on="utrip_id")

del(df_list)
gc.collect()

print(df_feat.shape)
display(df_feat)

(8855028, 83)


,utrip_id,city_id,score,label,rank,score_norm,user_id,checkin,checkout,device_class,affiliate_id,booker_country,total_rows,train,diff_total_rows,duration,same_country,checkin_day,checkin_weekday,checkin_week,checkin_month,checkin_year,checkin_day_of_year,checkout_week,checkout_day,checkin_week_sin,checkin_week_cos,time_elapsed_global,first_hotel_country,first_city_id,first_duration,first_same_country,first_affiliate_id,first_checkin,total_trip_duration,first_checkin_week,city_id_count,affiliate_id_count,hotel_country_count,user_id_count,checkin_week_count,checkin_day_of_year_count,city_id_nunique,hotel_country_nunique,affiliate_id_nunique,booker_country_nunique,duration_nunique,checkin_weekday_nunique,user_nunique_trips,city_id_rank_by_hotel_country,city_id_rank_by_affiliate,affiliate_id_rank_by_hotel_country,affiliate_id_rank_by_affiliate,hotel_country_rank_by_hotel_country,hotel_country_rank_by_affiliate,checkin_week_rank_by_hotel_country,checkin_week_rank_by_affiliate,city_id_rank_by_booker_country,lag1_city_id,lag1_affiliate_id,lag1_hotel_country,lag1_duration,lag1_same_country,lag1_checkin_week,lag1_city_id_count,lag1_affiliate_id_count,lag1_hotel_country_count,lag1_city_id_rank_by_hotel_country,lag1_city_id_rank_by_affiliate,lag2_city_id,lag2_affiliate_id,lag2_hotel_country,lag2_duration,lag2_same_country,lag2_checkin_week,lag2_city_id_count,lag2_affiliate_id_count,lag2_hotel_country_count,lag2_city_id_rank_by_hotel_country,lag2_city_id_rank_by_affiliate,lag3_city_id_count,lag3_city_id,lag3_hotel_country
0,1000027_1,65365,4.966359,0.0,0,0.611,1000027,232,235,0,253,1,4,1,0,3,0,18,3,33,8,1,231,33,21,-0.6068,-0.794854,231,60,8183,1,0,7168,227,5,32,93.0,4245,123562,4,66496,9562,4,1,2,1,3,4,1,50643.5,1337.0,20387.0,2123.0,61781.5,3253.0,105368.5,3913.0,97309.5,60902,7168,60,2.0,0.0,33.0,75.0,2509.0,123562.0,47455.0,949.0,15626,7168.0,60,2.0,0.0,32.0,1385.0,2509.0,123562.0,103049.0,2113.5,12.0,8183,60
1,1000027_1,60902,4.748976,0.0,1,0.622,1000027,232,235,0,253,1,4,1,0,3,0,18,3,33,8,1,231,33,21,-0.6068,-0.794854,231,60,8183,1,0,7168,227,5,32,93.0,4245,123562,4,66496,9562,4,1,2,1,3,4,1,50643.5,1337.0,20387.0,2123.0,61781.5,3253.0,105368.5,3913.0,97309.5,60902,7168,60,2.0,0.0,33.0,75.0,2509.0,123562.0,47455.0,949.0,15626,7168.0,60,2.0,0.0,32.0,1385.0,2509.0,123562.0,103049.0,2113.5,12.0,8183,60
2,1000027_1,58178,4.671995,0.0,2,0.635,1000027,232,235,0,253,1,4,1,0,3,0,18,3,33,8,1,231,33,21,-0.6068,-0.794854,231,60,8183,1,0,7168,227,5,32,93.0,4245,123562,4,66496,9562,4,1,2,1,3,4,1,50643.5,1337.0,20387.0,2123.0,61781.5,3253.0,105368.5,3913.0,97309.5,60902,7168,60,2.0,0.0,33.0,75.0,2509.0,123562.0,47455.0,949.0,15626,7168.0,60,2.0,0.0,32.0,1385.0,2509.0,123562.0,103049.0,2113.5,12.0,8183,60
3,1000027_1,58015,4.573554,0.0,3,0.640,1000027,232,235,0,253,1,4,1,0,3,0,18,3,33,8,1,231,33,21,-0.6068,-0.794854,231,60,8183,1,0,7168,227,5,32,93.0,4245,123562,4,66496,9562,4,1,2,1,3,4,1,50643.5,1337.0,20387.0,2123.0,61781.5,3253.0,105368.5,3913.0,97309.5,60902,7168,60,2.0,0.0,33.0,75.0,2509.0,123562.0,47455.0,949.0,15626,7168.0,60,2.0,0.0,32.0,1385.0,2509.0,123562.0,103049.0,2113.5,12.0,8183,60
4,1000027_1,5385,4.562560,0.0,4,0.653,1000027,232,235,0,253,1,4,1,0,3,0,18,3,33,8,1,231,33,21,-0.6068,-0.794854,231,60,8183,1,0,7168,227,5,32,93.0,4245,123562,4,66496,9562,4,1,2,1,3,4,1,50643.5,1337.0,20387.0,2123.0,61781.5,3253.0,105368.5,3913.0,97309.5,60902,7168,60,2.0,0.0,33.0,75.0,2509.0,123562.0,47455.0,949.0,15626,7168.0,60,2.0,0.0,32.0,1385.0,2509.0,123562.0,103049.0,2113.5,12.0,8183,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8855023,999991_3,33568,4.384465,NaN,45,0.845,999991,235,236,2,3631,1,4,0,0,1,0,21,6,33,8,1,234,34,22,-0.6068,-0.794854,234,46,29770,2,

### readd city/country specific features, for new candidates

* TODO: add time based features & counts (e.g. rank of city per date/week, etc')

* these features may overfit (or underfit for the "count" based). 

WARNING! BAD! WE are missing cities in the match after adding test data! 

In [38]:
df_feat.loc[df_feat["train"]==0].shape[0]

3533100

In [39]:
v1 = df_feat.shape[0]
v1

8855028

In [40]:
df_feat.loc[df_feat["train"]==0].merge(df_city_features,on="city_id",how="inner").shape[0]

3534795

In [41]:
df_feat.merge(df_city_features,on="city_id",how="inner").shape[0]

8859212

In [42]:
# ### fails ! IMPORTANT BUG! FIX! TODO ! For now we will ignore it..
# assert v1 == df_feat.merge(df_city_features,on="city_id",how="inner").shape[0]

In [43]:
%%time
df_feat = df_feat.drop(DROP_FEATS,axis=1,errors="ignore").merge(df_city_features,on="city_id",how="left")
# df_feat = df_feat.drop(DROP_FEATS,axis=1,errors="ignore").merge(df_city_features,on="city_id",how="inner")

Wall time: 16 s


In [44]:
%%time
df_feat["same_country"] = (df_feat["booker_country"]==df_feat["hotel_country"]).astype(int)

## combination features - could make more, normalize by counts of variables, etc' . + add it to categoricals
# df_feat["joint_city_lag1_city"] = df_feat["city_id"].astype(str)+"_"+df_feat["lag1_city_id"].astype(str)
df_feat["joint_city_count_city_lag1_count"] = df_feat["city_id_count"].astype(str)+"_"+df_feat["lag1_city_id_count"].astype(str)
# df_feat["joint_city_count_city_lag2_count"] = df_feat["city_id_count"].astype(str)+"_"+df_feat["lag2_city_id_count"].astype(str)

df_feat["joint_city_country_lag1_count"] = df_feat["city_id"].astype(str)+"_"+df_feat["lag1_hotel_country"].astype(str)

# df_feat["joint_city_first_city"] = df_feat["city_id_count"].astype(str)+"_"+df_feat["first_city_id"].astype(str)
# df_feat["joint_countries_first_lag1_current"] = df_feat["hotel_country"].astype(str)+"_"+df_feat["lag1_hotel_country"].astype(str)+"_"+df_feat["first_hotel_country"].astype(str)

### include the combination categoircal features added above:
ADDED_JOINT_FEAT_COLS = [#"joint_city_lag1_city",
                                   "joint_city_count_city_lag1_count",
#                          "joint_city_count_city_lag2_count",
                                   "joint_city_country_lag1_count",
#                                    "joint_city_first_city",
#                                    "joint_countries_first_lag1_current"                                  
                                  ]

for c in ADDED_JOINT_FEAT_COLS:
    df_feat[c] = map_rare_feats(df_feat,col=c,thresh=6,replace_val='')

df_feat[ADDED_JOINT_FEAT_COLS] = df_feat[ADDED_JOINT_FEAT_COLS].astype("category")

OBJECT_COLS = list(df_feat.select_dtypes("O").columns)
CAT_FEAT_NAMES = CAT_FEAT_NAMES + ADDED_JOINT_FEAT_COLS

# ["joint_city_count_city_lag1_count","joint_city_country_lag1_count"]

uniques 231982
afterwards uniques 152389
uniques 191888
afterwards uniques 44862
Wall time: 37.3 s


In [45]:
%%time
ord_encoder = OrdinalEncoder()
MINI_OBJ_COLS = [x for x in list(df_feat.select_dtypes("O").columns) if not x.endswith("id")]
df_feat[MINI_OBJ_COLS] = ord_encoder.fit_transform(df_feat[MINI_OBJ_COLS] )

Wall time: 573 ms


In [46]:
%%time
df_feat = df_feat.apply(pd.to_numeric, errors='ignore',downcast="float").apply(pd.to_numeric, errors='ignore',downcast="integer")

Wall time: 30.1 s


In [47]:
## check for feat A == feat B

df_feat["identical_hotel_country_l1"] = (df_feat["hotel_country"] == df_feat["lag1_hotel_country"]).astype(np.int8)
df_feat["identical_hotel_country_l2"] = (df_feat["hotel_country"] == df_feat["lag2_hotel_country"]).astype(np.int8)
df_feat["identical_hotel_country_first"] = (df_feat["hotel_country"] == df_feat["first_hotel_country"]).astype(np.int8)

df_feat["identical_city_id_l1"] = (df_feat["city_id"] == df_feat["lag1_city_id"]).astype(np.int8)
df_feat["identical_city_id_l2"] = (df_feat["city_id"] == df_feat["lag2_city_id"]).astype(np.int8)
df_feat["identical_city_id_first"] = (df_feat["city_id"] == df_feat["first_city_id"]).astype(np.int8)

df_feat["identical_affiliate_id_l1"] = (df_feat["affiliate_id"] == df_feat["lag1_affiliate_id"]).astype(np.int8)
df_feat["identical_affiliate_id_first"] = (df_feat["affiliate_id"] == df_feat["first_affiliate_id"]).astype(np.int8)

In [48]:
## for pool - values must be ordered by group

df_feat.sort_values(["utrip_id","label","checkin"],ascending=[True,False,True],inplace=True)

In [49]:
df_feat[OBJECT_COLS] = df_feat[OBJECT_COLS].fillna("")
df_feat[CAT_FEAT_NAMES] = df_feat[CAT_FEAT_NAMES].fillna(-1)

df_feat.dtypes

utrip_id                         object
city_id                           int32
score                           float32
label                           float32
rank                               int8
                                 ...   
identical_city_id_l1               int8
identical_city_id_l2               int8
identical_city_id_first            int8
identical_affiliate_id_l1          int8
identical_affiliate_id_first       int8
Length: 88, dtype: object

In [50]:
display(df_feat)

,utrip_id,city_id,score,label,rank,score_norm,user_id,checkin,checkout,device_class,affiliate_id,booker_country,total_rows,train,diff_total_rows,duration,checkin_day,checkin_weekday,checkin_week,checkin_month,checkin_year,checkin_day_of_year,checkout_week,checkout_day,checkin_week_sin,checkin_week_cos,time_elapsed_global,first_hotel_country,first_city_id,first_duration,first_same_country,first_affiliate_id,first_checkin,total_trip_duration,first_checkin_week,affiliate_id_count,user_id_count,checkin_week_count,checkin_day_of_year_count,affiliate_id_nunique,booker_country_nunique,duration_nunique,checkin_weekday_nunique,user_nunique_trips,checkin_week_rank_by_hotel_country,checkin_week_rank_by_affiliate,lag1_city_id,lag1_affiliate_id,lag1_hotel_country,lag1_duration,lag1_same_country,lag1_checkin_week,lag1_city_id_count,lag1_affiliate_id_count,lag1_hotel_country_count,lag1_city_id_rank_by_hotel_country,lag1_city_id_rank_by_affiliate,lag2_city_id,lag2_affiliate_id,lag2_hotel_country,lag2_duration,lag2_same_country,lag2_checkin_week,lag2_city_id_count,lag2_affiliate_id_count,lag2_hotel_country_count,lag2_city_id_rank_by_hotel_country,lag2_city_id_rank_by_affiliate,lag3_city_id_count,lag3_city_id,lag3_hotel_country,hotel_country,city_id_count,hotel_country_count,city_id_rank_by_hotel_country,numeric_hotel_country,numeric_city_id,same_country,joint_city_count_city_lag1_count,joint_city_country_lag1_count,identical_hotel_country_l1,identical_hotel_country_l2,identical_hotel_country_first,identical_city_id_l1,identical_city_id_l2,identical_city_id_first,identical_affiliate_id_l1,identical_affiliate_id_first
16,1000027_1,30628,4.025222,1.0,23,0.709,1000027,232,235,0,253,1,4,1,0,3,18,3,33,8,1,231,33,21,-0.6068,-0.794854,231,60,8183,1,0,7168,227,5,32,4245,4,66496,9562,2,1,3,4,1,105368.5,3913.0,60902,7168,60,2.0,0.0,33.0,75.0,2509.0,123562.0,47455.0,949.0,15626,7168.0,60,2.0,0.0,32.0,1385.0,2509.0,123562.0,103049.0,2113.5,12.0,8183,60,60,93,123562,50643.5,58,30628,0,93.0_75.0,30628_60,1,1,1,0,0,0,0,0
0,1000027_1,65365,4.966359,0.0,0,0.611,1000027,232,235,0,253,1,4,1,0,3,18,3,33,8,1,231,33,21,-0.6068,-0.794854,231,60,8183,1,0,7168,227,5,32,4245,4,66496,9562,2,1,3,4,1,105368.5,3913.0,60902,7168,60,2.0,0.0,33.0,75.0,2509.0,123562.0,47455.0,949.0,15626,7168.0,60,2.0,0.0,32.0,1385.0,2509.0,123562.0,103049.0,2113.5,12.0,8183,60,60,189,123562,63178.0,58,65365,0,189.0_75.0,65365_60,1,1,1,0,0,0,0,0
1,1000027_1,60902,4.748976,0.0,1,0.622,1000027,232,235,0,253,1,4,1,0,3,18,3,33,8,1,231,33,21,-0.6068,-0.794854,231,60,8183,1,0,7168,227,5,32,4245,4,66496,9562,2,1,3,4,1,105368.5,3913.0,60902,7168,60,2.0,0.0,33.0,75.0,2509.0,123562.0,47455.0,949.0,15626,7168.0,60,2.0,0.0,32.0,1385.0,2509.0,123562.0,103049.0,2113.5,12.0,8183,60,60,75,123562,47455.0,58,60902,0,75.0_75.0,60902_60,1,1,1,1,0,0,0,0
2,1000027_1,58178,4.671995,0.0,2,0.635,1000027,232,235,0,253,1,4,1,0,3,18,3,33,8,1,231,33,21,-0.6068,-0.794854,231,60,8183,1,0,7168,227,5,32,4245,4,66496,9562,2,1,3,4,1,105368.5,3913.0,60902,7168,60,2.0,0.0,33.0,75.0,2509.0,123562.0,47455.0,949.0,15626,7168.0,60,2.0,0.0,32.0,1385.0,2509.0,123562.0,103049.0,2113.5,12.0,8183,60,60,2842,123562,112552.5,58,58178,0,2842.0_75.0,58178_60,1,1,1,0,0,0,0,0
3,1000027_1,58015,4.573554,0.0,3,0.640,1000027,232,235,0,253,1,4,1,0,3,18,3,33,8,1,231,33,21,-0.6068,-0.794854,231,60,8183,1,0,7168,227,5,32,4245,4,66496,9562,2,1,3,4,1,105368.5,3913.0,60902,7168,60,2.0,0.0,33.0,75.0,2509.0,123562.0,47455.0,949.0,15626,7168.0,60,2.0,0.0,32.0,1385.0,2509.0,123562.0,103049.0,2113.5,12.0,8183,60,60,107,123562,53072.0,58,58015,0,107.0_75.0,58015_60,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8859207,999991_3,33568,4.384465

In [51]:
%%time
# df_feat = df_feat.apply(pd.to_numeric, errors='ignore',downcast="float").apply(pd.to_numeric, errors='ignore',downcast="integer")
df_feat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8859212 entries, 16 to 8859211
Data columns (total 88 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   utrip_id                            object 
 1   city_id                             int32  
 2   score                               float32
 3   label                               float32
 4   rank                                int8   
 5   score_norm                          float32
 6   user_id                             int32  
 7   checkin                             int16  
 8   checkout                            int16  
 9   device_class                        int8   
 10  affiliate_id                        int16  
 11  booker_country                      int8   
 12  total_rows                          int8   
 13  train                               int8   
 14  diff_total_rows                     int8   
 15  duration                            int8   
 16 

In [52]:
### data to get predictions on for contest
df_candidates = df_feat.loc[df_feat["train"]==0].drop(["train","label"],axis=1).copy()

df_feat = df_feat.loc[df_feat["train"]==1].drop(["train"],axis=1)#.copy()
print("prediction candidates shape",df_candidates.shape)
print("df_feat after keeping only labeled data: ",df_feat.shape)

prediction candidates shape (3534795, 86)
df_feat after keeping only labeled data:  (5324417, 87)


In [53]:
assert df_candidates.loc[df_candidates["city_id"].isna()].shape[0]==0

Save intermediate files

In [54]:
if SAVE_OUTPUTS:
    df_feat.to_parquet(path="df_merged_train.parquet",index=False)
    df_candidates.to_parquet(path="df_merged_candidates.parquet",index=False)

In [55]:
########################
## Groupwise train/test split by user or utrip_id

train_inds, test_inds = next(GroupShuffleSplit(test_size=.09,
                                               n_splits=2,
                                               random_state = 12).split(df_feat,groups=df_feat['utrip_id']))

X_train = df_feat.iloc[train_inds].drop([DROP_FEATS],axis=1,errors="ignore")#.merge(df_city_features,on="city_id",how="left")
X_test = df_feat.iloc[test_inds].drop([DROP_FEATS],axis=1,errors="ignore")#.merge(df_city_features,on="city_id",how="left")

y_train = X_train.pop(TARGET_COL)
y_test = X_test.pop(TARGET_COL)

print("# classes",y_train.nunique())

# ## check that same classes in train and test - 
# assert (set(y_train.unique()) == set(y_test.unique()))
assert y_train.isna().max() == y_test.isna().max() == 0

display(X_train)

# classes 2


,utrip_id,city_id,score,rank,score_norm,user_id,checkin,checkout,device_class,affiliate_id,booker_country,total_rows,diff_total_rows,duration,checkin_day,checkin_weekday,checkin_week,checkin_month,checkin_year,checkin_day_of_year,checkout_week,checkout_day,checkin_week_sin,checkin_week_cos,time_elapsed_global,first_hotel_country,first_city_id,first_duration,first_same_country,first_affiliate_id,first_checkin,total_trip_duration,first_checkin_week,affiliate_id_count,user_id_count,checkin_week_count,checkin_day_of_year_count,affiliate_id_nunique,booker_country_nunique,duration_nunique,checkin_weekday_nunique,user_nunique_trips,checkin_week_rank_by_hotel_country,checkin_week_rank_by_affiliate,lag1_city_id,lag1_affiliate_id,lag1_hotel_country,lag1_duration,lag1_same_country,lag1_checkin_week,lag1_city_id_count,lag1_affiliate_id_count,lag1_hotel_country_count,lag1_city_id_rank_by_hotel_country,lag1_city_id_rank_by_affiliate,lag2_city_id,lag2_affiliate_id,lag2_hotel_country,lag2_duration,lag2_same_country,lag2_checkin_week,lag2_city_id_count,lag2_affiliate_id_count,lag2_hotel_country_count,lag2_city_id_rank_by_hotel_country,lag2_city_id_rank_by_affiliate,lag3_city_id_count,lag3_city_id,lag3_hotel_country,hotel_country,city_id_count,hotel_country_count,city_id_rank_by_hotel_country,numeric_hotel_country,numeric_city_id,same_country,joint_city_count_city_lag1_count,joint_city_country_lag1_count,identical_hotel_country_l1,identical_hotel_country_l2,identical_hotel_country_first,identical_city_id_l1,identical_city_id_l2,identical_city_id_first,identical_affiliate_id_l1,identical_affiliate_id_first
16,1000027_1,30628,4.025222,23,0.709,1000027,232,235,0,253,1,4,0,3,18,3,33,8,1,231,33,21,-0.60680,-0.794854,231,60,8183,1,0,7168,227,5,32,4245,4,66496,9562,2,1,3,4,1,105368.5,3913.0,60902,7168,60,2.0,0.0,33.0,75.0,2509.0,123562.0,47455.0,949.0,15626,7168.0,60,2.0,0.0,32.0,1385.0,2509.0,123562.0,103049.0,2113.5,12.0,8183,60,60,93,123562,50643.5,58,30628,0,93.0_75.0,30628_60,1,1,1,0,0,0,0,0
0,1000027_1,65365,4.966359,0,0.611,1000027,232,235,0,253,1,4,0,3,18,3,33,8,1,231,33,21,-0.60680,-0.794854,231,60,8183,1,0,7168,227,5,32,4245,4,66496,9562,2,1,3,4,1,105368.5,3913.0,60902,7168,60,2.0,0.0,33.0,75.0,2509.0,123562.0,47455.0,949.0,15626,7168.0,60,2.0,0.0,32.0,1385.0,2509.0,123562.0,103049.0,2113.5,12.0,8183,60,60,189,123562,63178.0,58,65365,0,189.0_75.0,65365_60,1,1,1,0,0,0,0,0
1,1000027_1,60902,4.748976,1,0.622,1000027,232,235,0,253,1,4,0,3,18,3,33,8,1,231,33,21,-0.60680,-0.794854,231,60,8183,1,0,7168,227,5,32,4245,4,66496,9562,2,1,3,4,1,105368.5,3913.0,60902,7168,60,2.0,0.0,33.0,75.0,2509.0,123562.0,47455.0,949.0,15626,7168.0,60,2.0,0.0,32.0,1385.0,2509.0,123562.0,103049.0,2113.5,12.0,8183,60,60,75,123562,47455.0,58,60902,0,75.0_75.0,60902_60,1,1,1,1,0,0,0,0
2,1000027_1,58178,4.671995,2,0.635,1000027,232,235,0,253,1,4,0,3,18,3,33,8,1,231,33,21,-0.60680,-0.794854,231,60,8183,1,0,7168,227,5,32,4245,4,66496,9562,2,1,3,4,1,105368.5,3913.0,60902,7168,60,2.0,0.0,33.0,75.0,2509.0,123562.0,47455.0,949.0,15626,7168.0,60,2.0,0.0,32.0,1385.0,2509.0,123562.0,103049.0,2113.5,12.0,8183,60,60,2842,123562,112552.5,58,58178,0,2842.0_75.0,58178_60,1,1,1,0,0,0,0,0
3,1000027_1,58015,4.573554,3,0.640,1000027,232,235,0,253,1,4,0,3,18,3,33,8,1,231,33,21,-0.60680,-0.794854,231,60,8183,1,0,7168,227,5,32,4245,4,66496,9562,2,1,3,4,1,105368.5,3913.0,60902,7168,60,2.0,0.0,33.0,75.0,2509.0,123562.0,47455.0,949.0,15626,7168.0,60,2.0,0.0,32.0,1385.0,2509.0,123562.0,103049.0,2113.5,12.0,8183,60,60,107,123562,53072.0,58,58015,0,107.0_75.0,58015_60,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3194810,999944_1,9590,4.665932,23,0.822,999944,182,183,0,4541,2,4,0,1,29,2,2

In [56]:
### delete df_feat, save on memory. (some commented out processing code assumes its existence though! )
del df_feat
gc.collect()


380

## Model
* For now - simple multiclass model (Tabnet? LSTM?) ; +- subsample - only most frequent classes/cities

    * Tabnet: `pip install pytorch-tabnet`
        * https://github.com/dreamquark-ai/tabnet/blob/develop/forest_example.ipynb
    * TensorFlow Tabmet: https://github.com/ostamand/tensorflow-tabnet/blob/master/examples/train_mnist.py

* split train/test by user id. 
    * Test - only last trip. 
    
* Try multiclass models

* Try tabnet models (tabular with attention)
    * + Lag feats
    * Note - the embedding here is not aware that the same IDs are the same (unlike TF's )! 

* cat_idxs : list of int (default=[] - Mandatory for embeddings)
    * List of categorical features indices.

* cat_dims : list of int (default=[] - Mandatory for embeddings)

    * List of categorical features number of modalities (number of unique values for a categorical feature) /!\ no new modalities can be predicted

* cat_emb_dim : list of int (optional)

    * List of embeddings size for each categorical features. (default =1)
    
    
    
All the categorical vals must be known from train (demo used label encoder). Consider doing so also here at late step, to avoid unknown vals ? 

In [57]:
print(CAT_FEAT_NAMES)

['booker_country', 'device_class', 'affiliate_id', 'city_id', 'hotel_country', 'checkin_week', 'lag1_city_id', 'lag1_hotel_country', 'lag1_affiliate_id', 'lag2_city_id', 'lag2_hotel_country', 'lag3_city_id', 'lag3_hotel_country', 'first_hotel_country', 'first_city_id', 'joint_city_count_city_lag1_count', 'joint_city_country_lag1_count']


In [58]:
# df_feat[OBJECT_COLS] = df_feat[OBJECT_COLS].fillna("")
X_train[OBJECT_COLS] = X_train[OBJECT_COLS].fillna("")
X_test[OBJECT_COLS] = X_test[OBJECT_COLS].fillna("")

df_candidates[OBJECT_COLS] = df_candidates[OBJECT_COLS].fillna("")

In [59]:
# df_feat[CAT_FEAT_NAMES] = df_feat[CAT_FEAT_NAMES].fillna(-1)
X_train[CAT_FEAT_NAMES] = X_train[CAT_FEAT_NAMES].fillna(-1)
X_test[CAT_FEAT_NAMES] = X_test[CAT_FEAT_NAMES].fillna(-1)

df_candidates[CAT_FEAT_NAMES] = df_candidates[CAT_FEAT_NAMES].fillna(-1)

Ordinal/categoircals encoder

In [60]:
assert X_test[CAT_FEAT_NAMES].isna().sum().max() == X_train[CAT_FEAT_NAMES].isna().sum().max() == 0

#### feature importance & evaluation
* Look for leaks!
* May be bug with ordering of results - evaluation doesn't make sense. Note that diff # outputs/classes, likely culprit

# Catboost Classification/Ranking model
* +- add embeddings from lightfm, more features etc' 

* Catboost has ranking models. And supports embedding features. And taking time into account.
    * Example catboost ranking models/objectives: https://catboost.ai/docs/concepts/loss-functions-ranking.html
    * We could put in previous positive places visited in a trip as "weak positives" (e.g. "0.5 score" or a relative score based on distance/steps from final destination) 

In [61]:
X_train[CAT_FEAT_NAMES] = X_train[CAT_FEAT_NAMES].apply(pd.to_numeric, errors='ignore',downcast="integer")
X_test[CAT_FEAT_NAMES] = X_test[CAT_FEAT_NAMES].apply(pd.to_numeric, errors='ignore',downcast="integer")
df_candidates[CAT_FEAT_NAMES] = df_candidates[CAT_FEAT_NAMES].apply(pd.to_numeric, errors='ignore',downcast="integer")

X_train[CAT_FEAT_NAMES].dtypes

booker_country                        int8
device_class                          int8
affiliate_id                         int16
city_id                              int32
hotel_country                        int16
checkin_week                          int8
lag1_city_id                         int32
lag1_hotel_country                   int16
lag1_affiliate_id                    int16
lag2_city_id                         int32
lag2_hotel_country                   int16
lag3_city_id                         int32
lag3_hotel_country                   int16
first_hotel_country                  int16
first_city_id                        int32
joint_city_count_city_lag1_count    object
joint_city_country_lag1_count       object
dtype: object

In [62]:
print("train rows",X_train.shape[0])
print("test rows",X_test.shape[0])

train rows 4845197
test rows 479220


In [63]:
def top4_accuracy(model,y_test_pred=None):
    """warning - very hacky! - set to use global X_test, y_test variables"""
    if y_test_pred is None:
        y_test_pred = model.predict(X_test,prediction_type="Probability")[:,1]

    test_preds = pd.DataFrame({"y_test_pred":y_test_pred,
#                                "candidate_city_id":X_test["city_id"],
                               "utrip_id":X_test["utrip_id"],
                               "rank":X_test["rank"],
#                                "city_id_count":X_test["city_id_count"],
                               "label":y_test
                              })    
    ## most frequent in overall data (not as target)
    n_users = test_preds["utrip_id"].nunique()

#     test_preds.sort_values(["utrip_id","city_id_count"],inplace=True,ascending=False)
#     res_count = 100*test_preds.groupby("utrip_id")["label"].head(4).sum() / n_users
#     print("top4 acc baseline: by most frequent city_id_count {0:.4f}".format(res_count))

    test_preds.sort_values(["utrip_id","rank"],inplace=True,ascending=True)
    res_rank = 100*test_preds.groupby("utrip_id")["label"].head(4).sum() / n_users
    print("top4 acc baseline: by rank {0:.4f}".format(res_rank))

    test_preds.sort_values(["utrip_id","y_test_pred"],inplace=True,ascending=False)
    res = 100*test_preds.groupby("utrip_id")["label"].head(4).sum() / n_users
    print("top4 acc: by model {0:.4f}".format(res))
    return res

## Catboost model

In [64]:
# ## yield splits for groupwise CV (within train) - do here before deleting x train
# group_cv_train = GroupShuffleSplit(n_splits=3)#GroupKFold(n_splits=3)
# group_cv = group_cv_train.split(X_train,y_train,groups=X_train["utrip_id"])

In [65]:
X_test["score"].isna().sum()

0

In [66]:
%%time
##TEMP: disable pool, save on memory! 
# X_train.drop(["score","rank","score_norm"],axis=1)
train_pool = Pool(data=X_train,#.drop(["score","rank","score_norm"],axis=1),
                  label = y_train,cat_features=CAT_FEAT_NAMES
#                  ,group_id=X_train["utrip_id"]
                 )
print("Train pool done")
del X_train
gc.collect()

test_pool = Pool(data=X_test,
                 label = y_test,cat_features=CAT_FEAT_NAMES
#                 ,group_id=X_test["utrip_id"]
                )

Train pool done
Wall time: 1min 54s


In [67]:
# https://colab.research.google.com/github/catboost/tutorials/blob/master/ranking/ranking_tutorial.ipynb#scrollTo=PC-BWnUscjHB

default_parameters = {
    'iterations': max_epochs,
#     'custom_metric': ['AUC:type=Ranking', 'RecallAt:top=4'],
    'verbose': False,
    'random_seed': 0,
    "metric_period":20,
    "task_type":"GPU",
#     'loss_function': "PairLogit", # "PairLogitPairwise"
#     "ignored_features":["utrip_id"],
     "eval_metric":'AUC',
#     'loss_function':"AUC",
    "auto_class_weights":"SqrtBalanced",

}

## https://catboost.ai/docs/concepts/parameter-tuning.html#border-count
## per_float_feature_quantization=['0:border_count=1024', '1:border_count=1024']

parameters = {}
parameters = deepcopy(default_parameters)

### parameter grid for grid search / Hyperparameter tuning:
# hyper_params = {'depth':[4,6,8,10],
#           'l2_leaf_reg':[3,1,5,10],
# #           'border_count':[32,64,128,254], ## we may want to set city ID as golden features
# #         "auto_class_weights":[None,"SqrtBalanced","Balanced"],
# #           'ctr_border_count':[50,5,10,20,100,200],
#                 "min_data_in_leaf":[1,2],
#          }

hyper_params = {'depth':[5,8],
          'l2_leaf_reg':[3,7],
#           'border_count':[32,64,128,254], ## we may want to set city ID as golden features
#         "auto_class_weights":[None,"SqrtBalanced","Balanced"],
#           'ctr_border_count':[50,5,10,20,100,200],
#                 "min_data_in_leaf":[1,2], # not used in symetric trees - https://heartbeat.fritz.ai/fast-gradient-boosting-with-catboost-38779b0d5d9a         
               }

#          'loss_function':["PairLogit", "PairLogitPairwise","YetiRank"], # not supported in grid search :()


### Hyperparameter search 
* Run in pipeline on subset/sample of data
* CatBoostError: Parameter 'loss_function' currently is not supported in grid search :(

* try setting golden features? 
    * https://catboost.ai/docs/concepts/parameter-tuning.html#golden-features
    * `per_float_feature_quantization=['0:border_count=1024', '1:border_count=1024']`
    
    * `max_ctr_complexity`  (default : 4 )
        * https://catboost.ai/docs/concepts/python-reference_parameters-list.html
* TODO : try model with everything converted to ordinals (instead of categoricals handling)

In [102]:
# pd.DataFrame(grid_search_result["cv_results"]).tail()
# ## 160k samples, 400 epochs, golden qunatization, test-AUC:type=Ranking-mean: 0.67, test-Logloss-mean: 0.633

In [103]:
# model = CatBoostClassifier()
# model.load_model("trained_model")

In [ ]:
### Normal classifier (not ranking) :
model = CatBoostClassifier(verbose=True,
                           iterations=max_epochs,
#                            iterations=1000,
                           eval_metric='AUC',
                           task_type="GPU",
                           metric_period=30,

                          ignored_features=["utrip_id"],
                          auto_class_weights="SqrtBalanced",
                           cat_features=CAT_FEAT_NAMES,                           
                             od_type = "Iter",
                              od_wait = 40,
#                            max_ctr_complexity=3, # default 4
                           gpu_ram_part = 0.88,
                          )

###orig:
model.fit(train_pool,  eval_set=test_pool, plot=False, 
          use_best_model=False,
         )

### alt - without pool


# model.fit(X_train.drop(["score","rank","score_norm"],axis=1),y_train,  eval_set=(X_test.drop(["score","rank","score_norm"],axis=1),y_test), plot=False,        
# #           silent=True,
#           use_best_model=True,
# #          save_snapshot=True
#          )


print(model.get_best_score())

# # Top 4 Accuracy: 56.5856
# {'learn': {'Logloss': 0.047, 'AUC': 0.967}, 
#  'validation': {'Logloss': 0.0764, 'AUC': 0.839}}

# ## withoyut golden feats per_float_feature_quantization. small sample 
# bestTest = 0.9282451868
# bestIteration = 249
# {'learn': {'Logloss': 0.12, 'AUC': 0.956}, 'validation': {'Logloss': 0.15, 'AUC': 0.928}}

# top4 acc baseline: by most frequent city_id_count 13.3
# top4 acc baseline: by rank 37.3
# top4 acc: by model 49.33


# ### 
# {'learn': {'Logloss': 0.210, 'AUC': 0.934,
#'validation': {'Logloss': 0.2657, 'AUC': 0.869}}


# ## samplke, with rank feature : 
# bestTest = 0.8669822216
# bestIteration = 199
# {'learn': {'Logloss': 0.229, 'AUC': 0.91504}, 'validation': {'Logloss': 0.271, 'AUC': 0.8669}}

## without the score/rank, score is about 2-4 auc % lower
# 'validation': {'Logloss': 0.29, 'AUC': 0.84}}

Learning rate set to 0.021139
0:	learn: 0.6856391	test: 0.7114435	best: 0.7114435 (0)	total: 1.52s	remaining: 25m 15s
30:	learn: 0.7043832	test: 0.7270773	best: 0.7277890 (23)	total: 42.4s	remaining: 22m 3s
60:	learn: 0.7337425	test: 0.7555021	best: 0.7555021 (60)	total: 1m 24s	remaining: 21m 39s
90:	learn: 0.7717580	test: 0.7855440	best: 0.7855440 (90)	total: 2m 9s	remaining: 21m 30s
120:	learn: 0.7840100	test: 0.7947882	best: 0.7947882 (120)	total: 2m 54s	remaining: 21m 9s
150:	learn: 0.7936463	test: 0.8028627	best: 0.8028627 (150)	total: 3m 40s	remaining: 20m 42s
180:	learn: 0.8209962	test: 0.8069514	best: 0.8069514 (180)	total: 4m 27s	remaining: 20m 11s
210:	learn: 0.8730706	test: 0.8079712	best: 0.8079712 (210)	total: 5m 12s	remaining: 19m 26s
240:	learn: 0.8891786	test: 0.8086583	best: 0.8086583 (240)	total: 5m 55s	remaining: 18m 39s
270:	learn: 0.8967516	test: 0.8092331	best: 0.8093434 (266)	total: 6m 40s	remaining: 17m 56s
300:	learn: 0.9010060	test: 0.8103332	best: 0.8103332 (

In [ ]:
model.get_all_params()

In [ ]:
## wit hauc, sqrt balanced
# 180:	learn: 0.9281184	test: 0.7967654	best: 0.7982976 (144)	total: 44.5s	remaining: 2m 56s
# 200:	learn: 0.9340843	test: 0.7858477	best: 0.7982976 (144)	total: 49.6s	remaining: 2m 52s
# bestTest = 0.7982975841
# bestIteration = 144
# Shrink model to first 145 iterations.
# {'learn': {'Logloss': 0.22825456382950968, 'AUC': 0.9340843260288239}, 'validation': {'Logloss': 0.33421671828444016, 'AUC': 0.7982975840568542}}

In [ ]:
# # print()
# top4_acc = top4_accuracy(model)

# ## on 52 k fst sample: 
# # top4 acc baseline: by most frequent city_id_count 23.0867
# # top4 acc baseline: by rank 42.7296
# # top4 acc: by model 48.9796

#### Get predictions on competition/candidate data
* `df_candidates`
* There appear to be duplicate city_ids in the predictions ? 

In [ ]:
print(df_candidates.shape)
assert df_candidates.loc[df_candidates["city_id"].isna()].shape[0] ==0

In [ ]:
# df_test["utrip_id"].nunique() - df_candidates["utrip_id"].nunique()

In [ ]:
df_candidates

In [ ]:
SAVE_OUTPUTS = True

In [ ]:
%%time
if SAVE_OUTPUTS:
    candidates_pool = Pool(data=df_candidates,cat_features=CAT_FEAT_NAMES
#                      ,group_id=df_candidates["utrip_id"]
                     )
    print("candidate pool done")
    preds = model.predict(candidates_pool,prediction_type="Probability")[:,1]

In [ ]:
# %%time
# # errors if using raw data - data type disparities . unclear why
# ## thread_count
# preds = model.predict(df_candidates,prediction_type="Probability",verbose=True,thread_count=4)[:,1]
# print("done")
# print(len(preds))

In [ ]:
if SAVE_OUTPUTS:
    submission = pd.DataFrame({
                               "utrip_id":df_candidates["utrip_id"],
                           "city_id":df_candidates["city_id"],
                           "pred":preds,
    #                        "rank":df_candidates["rank"],
                          })   

    submission.sort_values(["pred"],ascending=False,inplace=True)

    ### unknonw bug - we get multiple cities per ID ???

    print("duplicate cities suggested:",submission.shape[0] - submission.drop_duplicates(['utrip_id','city_id']).shape[0])

    #  if SAVE_OUTPUTS:
    submission = submission.drop_duplicates(['utrip_id','city_id'],keep="first")
    print("submission.nunique()",submission.nunique())
    display(submission)
    ## drop -1s:
    submission = submission.loc[submission['city_id']>= 0]
    submission = submission.groupby(["utrip_id"]).head(4).drop("pred",axis=1)

    submission["r_num"] = submission.groupby("utrip_id")["city_id"].rank(ascending=True,pct=False)

    submission = submission.pivot(index='utrip_id', columns='r_num', values='city_id').reset_index() ## , values='score'
    submission.columns = ["utrip_id",'city_id_1','city_id_2','city_id_3','city_id_4']

    assert submission["utrip_id"].nunique() == test_num_ids
    display(submission)       

    submission.to_csv("submission.csv",index=False)
    print(submission.nunique())
    print(submission.shape)

In [ ]:
## gets stuck with many rows

# y_test_preds = model.predict(test_pool,prediction_type="Probability")[:,1]
# print(top4_accuracy(model,y_test_preds))

### from run on sample, with old train-train lightfm ranking: 
# top4 acc baseline: by most frequent city_id_count 39.6007
# top4 acc baseline: by rank 41.2646
# top4 acc: by model 77.0383
# 77.03826955074875

# ## still ~ 160k sample , model without rank/score: 
# top4 acc baseline: by most frequent city_id_count 32.1688
# top4 acc baseline: by rank 41.6647
# top4 acc: by model 49.9179

## Feature importance
* if usingSHAP: We don't want to wait too long/to calculate it over 40 million rows, so we could look at a sample. e.g. top ranked (rank <= 10 | label ==1)  , or first few rows


In [ ]:
top4_acc = top4_accuracy(model)
# # print(top4_accuracy(model))

# ## without sqrt balance, top4 acc: by model 56.9870

In [ ]:
# feat_imp_cb = model.get_feature_importance(data=test_pool,
#                        prettified=True,shap_calc_type ="Approximate")

# feat_imp_cb = feat_imp_cb.loc[feat_imp_cb["Importances"]>0].round(2)
# display(feat_imp_cb.head(32))

###### Evaluate model on test data
    * function may have memory leakage issue

* Using 30% of data:
    * 
        top4 acc baseline: by most frequent city_id_count 23.99
        
        top4 acc baseline: by rank 41.87
        
        top4 acc: by model 55.13